In [4]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pylab as pl
from IPython import display
import sys
%matplotlib inline

In [5]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

In [6]:
mnist = read_data_sets("data", one_hot=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
class RNN_cell(object):

    """
    RNN cell object which takes 3 arguments for initialization.
    input_size = Input Vector size
    hidden_layer_size = Hidden layer size
    target_size = Output vector size

    """

    def __init__(self, input_size, hidden_layer_size, target_size):

        #Initialization of given values
        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.target_size = target_size
        
        # Weights for input and hidden tensor
        self.Wx = tf.Variable(tf.zeros([self.input_size,self.hidden_layer_size]))
        self.Wr = tf.Variable(tf.zeros([self.input_size,self.hidden_layer_size]))
        self.Wz = tf.Variable(tf.zeros([self.input_size,self.hidden_layer_size]))
        
        self.br = tf.Variable(tf.truncated_normal([self.hidden_layer_size],mean=1))
        self.bz = tf.Variable(tf.truncated_normal([self.hidden_layer_size],mean=1))
        
        self.Wh = tf.Variable(tf.zeros([self.hidden_layer_size,self.hidden_layer_size]))

        
        #Weights for output layer
        self.Wo = tf.Variable(tf.truncated_normal([self.hidden_layer_size,self.target_size],mean=1,stddev=.01))
        self.bo = tf.Variable(tf.truncated_normal([self.target_size],mean=1,stddev=.01))
        # Placeholder for input vector with shape[batch, seq, embeddings]
        self._inputs = tf.placeholder(tf.float32,
                                      shape=[None, None, self.input_size],
                                      name='inputs')

        # Processing inputs to work with scan function
        self.processed_input = process_batch_input_for_RNN(self._inputs)

        '''
        Initial hidden state's shape is [1,self.hidden_layer_size]
        In First time stamp, we are doing dot product with weights to
        get the shape of [batch_size, self.hidden_layer_size].
        For this dot product tensorflow use broadcasting. But during
        Back propagation a low level error occurs.
        So to solve the problem it was needed to initialize initial
        hiddden state of size [batch_size, self.hidden_layer_size].
        So here is a little hack !!!! Getting the same shaped
        initial hidden state of zeros.
        '''

        self.initial_hidden = self._inputs[:, 0, :]
        self.initial_hidden = tf.matmul(
            self.initial_hidden, tf.zeros([input_size, hidden_layer_size]))
        
        
    #Function for GRU cell
    def Gru(self, previous_hidden_state, x):
        """
        GRU Equations
        """
        z= tf.sigmoid(tf.matmul(x,self.Wz)+ self.bz)
        r= tf.sigmoid(tf.matmul(x,self.Wr)+ self.br)
        
        h_= tf.tanh(tf.matmul(x,self.Wx) + tf.matmul(previous_hidden_state,self.Wh)*r)
                    
        
        current_hidden_state = tf.multiply((1-z),h_) + tf.multiply(previous_hidden_state,z)
        
        return current_hidden_state     
    
    # Function for getting all hidden state.
    def get_states(self):
        """
        Iterates through time/ sequence to get all hidden state
        """

        # Getting all hidden state throuh time
        all_hidden_states = tf.scan(self.Gru,
                                    self.processed_input,
                                    initializer=self.initial_hidden,
                                    name='states')

        return all_hidden_states

    # Function to get output from a hidden layer
    def get_output(self, hidden_state):
        """
        This function takes hidden state and returns output
        """
        output = tf.nn.relu(tf.matmul(hidden_state, self.Wo) + self.bo)

        return output

    # Function for getting all output layers
    def get_outputs(self):
        """
        Iterating through hidden states to get outputs for all timestamp
        """
        all_hidden_states = self.get_states()

        all_outputs = tf.map_fn(self.get_output, all_hidden_states)

        return all_outputs


# Function to convert batch input data to use scan ops of tensorflow.P
def process_batch_input_for_RNN(batch_input):
    """
    Process tensor of size [5,3,2] to [3,5,2]
    """
    batch_input_ = tf.transpose(batch_input, perm=[2, 0, 1])
    X = tf.transpose(batch_input_)

    return X

In [8]:
hidden_layer_size = 30
input_size = 784
target_size = 10
batch_size = 10
epoch = 15

In [9]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

W = tf.Variable(tf.random_normal([100, 10]))
b = tf.Variable(tf.random_normal([10]))

In [10]:
y = tf.placeholder(tf.float32, shape=[None, target_size],name='inputs')

In [11]:
#Initializing rnn object
rnn=RNN_cell( input_size, hidden_layer_size, target_size)

In [13]:
rnn.Gru(100, X)

TypeError: Input 'b' of 'MatMul' Op has type float32 that does not match type int32 of argument 'a'.

In [6]:
#Getting all outputs from rnn
outputs = rnn.get_outputs()

In [7]:
#Getting final output through indexing after reversing
last_output = outputs[-1]

In [8]:
#As rnn model output the final layer through Relu activation softmax is used for final output.
output=tf.nn.softmax(last_output)

In [28]:
#Computing the Cross Entropy loss 
cost = -tf.reduce_sum(y * tf.log(output))

In [26]:
# Trainning with Adadelta Optimizer
train = tf.train.AdamOptimizer().minimize(cross_entropy)

In [11]:
#Calculatio of correct prediction and accuracy
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(output,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100

In [12]:
sess = tf.Session()

In [13]:
sess.run(tf.global_variables_initializer())

In [29]:
total_batch = int(mnist.train.num_examples/batch_size)
for e in range(epoch):
    total_cost = 0
    print(e+1)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, 28, 28))

        _, _cost = sess.run([train, cost],feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += _cost
    test_batch_size = len(mnist.test.images)
    a = sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(test_batch_size, 28, 28), Y:mnist.test.labels})
    print(a)

1


FailedPreconditionError: Attempting to use uninitialized value beta1_power_1
	 [[Node: beta1_power_1/read = Identity[T=DT_FLOAT, _class=["loc:@Adam_1/Assign_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](beta1_power_1)]]

Caused by op 'beta1_power_1/read', defined at:
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\site-packages\ipykernel\kernelapp.py", line 497, in start
    self.io_loop.start()
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\site-packages\tornado\platform\asy